<h1><font color='brown'>Naive Bayes Part 1: Predicting survival from titanic crash</font></h1>

In [4]:
import pandas as pd
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [5]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [8]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [ ]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [9]:
dummies=pd.get_dummies(inputs.Sex)
dummies.head(2)

,female,male
0,0,1
1,1,0


In [12]:
inputs=pd.concat([inputs, dummies], axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male,female,male
0,3,male,22.0,7.2500,0,1,0,1
1,1,female,38.0,71.2833,1,0,1,0
2,3,female,26.0,7.9250,1,0,1,0


<h3>I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female</h3>

In [14]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female,female
0,3,22.0,7.2500,0,0
1,1,38.0,71.2833,1,1
2,3,26.0,7.9250,1,1


In [15]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [16]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [17]:
inputs.Age= inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female,female
0,3,22.0,7.2500,0,0
1,1,38.0,71.2833,1,1
2,3,26.0,7.9250,1,1
3,1,35.0,53.1000,1,1
4,3,35.0,8.0500,0,0


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [19]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()

In [28]:
model.fit(X_train, y_train).get_params()


{'priors': None, 'var_smoothing': 1e-09}

In [29]:
model.score(X_test,y_test)

0.7723880597014925

In [30]:
X_test[0:10]

,Pclass,Age,Fare,female,female
38,3,18.000000,18.0000,1,1
208,3,16.000000,7.7500,1,1
510,3,29.000000,7.7500,0,0
58,2,5.000000,27.7500,1,1
621,1,42.000000,52.5542,0,0
386,3,1.000000,46.9000,0,0
792,3,29.699118,69.5500,1,1
79,3,30.000000,12.4750,1,1
232,2,59.000000,13.5000,0,0
40,3,40.000000,9.4750,1,1


In [31]:
y_test[0:10]

38     0
208    1
510    1
58     1
621    1
386    0
792    0
79     1
232    0
40     0
Name: Survived, dtype: int64

In [32]:
model.predict(X_test[0:10])

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1])

In [33]:
model.predict_proba(X_test[:10])

array([[0.02102035, 0.97897965],
       [0.02091302, 0.97908698],
       [0.98751806, 0.01248194],
       [0.0060152 , 0.9939848 ],
       [0.87102269, 0.12897731],
       [0.95901996, 0.04098004],
       [0.01023855, 0.98976145],
       [0.02633993, 0.97366007],
       [0.96515727, 0.03484273],
       [0.02726342, 0.97273658]])

Calculate the score using cross validation

In [34]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.8       , 0.824     , 0.768     , 0.72580645, 0.80645161])